In [1]:
using Random, Printf
using DifferentialEquations
using Flux, DiffEqFlux

In [2]:
# The POLLU problem described in 
# https://benchmarks.sciml.ai/html/StiffODE/Pollution.html
k0 = [
    .35e0,   .266e2,  .123e5,  .86e-3,  .82e-3,  .15e5,
    .13e-3,  .24e5,   .165e5,  .9e4,    .22e-1,  .12e5,
    .188e1,  .163e5,  .48e7,   .35e-3,  .175e-1, .1e9,
    .444e12, .124e4,  .21e1,   .578e1,  .474e-1, .178e4,
    .312e1
];

In [3]:
function f(dy,y,p,t)
    k = k0 .* p;
    r1  = k[1] *y[1]
    r2  = k[2] *y[2]*y[4]
    r3  = k[3] *y[5]*y[2]
    r4  = k[4] *y[7]
    r5  = k[5] *y[7]
    r6  = k[6] *y[7]*y[6]
    r7  = k[7] *y[9]
    r8  = k[8] *y[9]*y[6]
    r9  = k[9] *y[11]*y[2]
    r10 = k[10]*y[11]*y[1]
    r11 = k[11]*y[13]
    r12 = k[12]*y[10]*y[2]
    r13 = k[13]*y[14]
    r14 = k[14]*y[1]*y[6]
    r15 = k[15]*y[3]
    r16 = k[16]*y[4]
    r17 = k[17]*y[4]
    r18 = k[18]*y[16]
    r19 = k[19]*y[16]
    r20 = k[20]*y[17]*y[6]
    r21 = k[21]*y[19]
    r22 = k[22]*y[19]
    r23 = k[23]*y[1]*y[4]
    r24 = k[24]*y[19]*y[1]
    r25 = k[25]*y[20]

    dy[1]  = -r1-r10-r14-r23-r24+
              r2+r3+r9+r11+r12+r22+r25
    dy[2]  = -r2-r3-r9-r12+r1+r21
    dy[3]  = -r15+r1+r17+r19+r22
    dy[4]  = -r2-r16-r17-r23+r15
    dy[5]  = -r3+r4+r4+r6+r7+r13+r20
    dy[6]  = -r6-r8-r14-r20+r3+r18+r18
    dy[7]  = -r4-r5-r6+r13
    dy[8]  =  r4+r5+r6+r7
    dy[9]  = -r7-r8
    dy[10] = -r12+r7+r9
    dy[11] = -r9-r10+r8+r11
    dy[12] =  r9
    dy[13] = -r11+r10
    dy[14] = -r13+r12
    dy[15] =  r14
    dy[16] = -r18-r19+r16
    dy[17] = -r20
    dy[18] =  r20
    dy[19] = -r21-r22-r24+r23+r25
    dy[20] = -r25+r24
end

f (generic function with 1 method)

In [4]:
function fjac(J,y,p,t)
    k = k0 .* p;
    J .= 0.0
    J[1,1]   = -k[1]-k[10]*y[11]-k[14]*y[6]-k[23]*y[4]-k[24]*y[19]
    J[1,11]  = -k[10]*y[1]+k[9]*y[2]
    J[1,6]   = -k[14]*y[1]
    J[1,4]   = -k[23]*y[1]+k[2]*y[2]
    J[1,19]  = -k[24]*y[1]+k[22]
    J[1,2]   = k[2]*y[4]+k[9]*y[11]+k[3]*y[5]+k[12]*y[10]
    J[1,13]  = k[11]
    J[1,20]  = k[25]
    J[1,5]   = k[3]*y[2]
    J[1,10]  = k[12]*y[2]

    J[2,4]   = -k[2]*y[2]
    J[2,5]   = -k[3]*y[2]
    J[2,11]  = -k[9]*y[2]
    J[2,10]  = -k[12]*y[2]
    J[2,19]  = k[21]
    J[2,1]   = k[1]
    J[2,2]   = -k[2]*y[4]-k[3]*y[5]-k[9]*y[11]-k[12]*y[10]

    J[3,1]   = k[1]
    J[3,4]   = k[17]
    J[3,16]  = k[19]
    J[3,19]  = k[22]
    J[3,3]   = -k[15]

    J[4,4]   = -k[2]*y[2]-k[16]-k[17]-k[23]*y[1]
    J[4,2]   = -k[2]*y[4]
    J[4,1]   = -k[23]*y[4]
    J[4,3]   = k[15]

    J[5,5]   = -k[3]*y[2]
    J[5,2]   = -k[3]*y[5]
    J[5,7]   = 2k[4]+k[6]*y[6]
    J[5,6]   = k[6]*y[7]+k[20]*y[17]
    J[5,9]   = k[7]
    J[5,14]  = k[13]
    J[5,17]  = k[20]*y[6]

    J[6,6]   = -k[6]*y[7]-k[8]*y[9]-k[14]*y[1]-k[20]*y[17]
    J[6,7]   = -k[6]*y[6]
    J[6,9]   = -k[8]*y[6]
    J[6,1]   = -k[14]*y[6]
    J[6,17]  = -k[20]*y[6]
    J[6,2]   = k[3]*y[5]
    J[6,5]   = k[3]*y[2]
    J[6,16]  = 2k[18]

    J[7,7]   = -k[4]-k[5]-k[6]*y[6]
    J[7,6]   = -k[6]*y[7]
    J[7,14]  = k[13]

    J[8,7]   = k[4]+k[5]+k[6]*y[6]
    J[8,6]   = k[6]*y[7]
    J[8,9]   = k[7]

    J[9,9]   = -k[7]-k[8]*y[6]
    J[9,6]   = -k[8]*y[9]

    J[10,10] = -k[12]*y[2]
    J[10,2]  = -k[12]*y[10]+k[9]*y[11]
    J[10,9]  = k[7]
    J[10,11] = k[9]*y[2]

    J[11,11] = -k[9]*y[2]-k[10]*y[1]
    J[11,2]  = -k[9]*y[11]
    J[11,1]  = -k[10]*y[11]
    J[11,9]  = k[8]*y[6]
    J[11,6]  = k[8]*y[9]
    J[11,13] = k[11]

    J[12,11] = k[9]*y[2]
    J[12,2]  = k[9]*y[11]

    J[13,13] = -k[11]
    J[13,11] = k[10]*y[1]
    J[13,1]  = k[10]*y[11]

    J[14,14] = -k[13]
    J[14,10] = k[12]*y[2]
    J[14,2]  = k[12]*y[10]

    J[15,1]  = k[14]*y[6]
    J[15,6]  = k[14]*y[1]

    J[16,16] = -k[18]-k[19]
    J[16,4]  = k[16]

    J[17,17] = -k[20]*y[6]
    J[17,6]  = -k[20]*y[17]

    J[18,17] = k[20]*y[6]
    J[18,6]  = k[20]*y[17]

    J[19,19] = -k[21]-k[22]-k[24]*y[1]
    J[19,1]  = -k[24]*y[19]+k[23]*y[4]
    J[19,4]  = k[23]*y[1]
    J[19,20] = k[25]

    J[20,20] = -k[25]
    J[20,1]  = k[24]*y[19]
    J[20,19] = k[24]*y[1]

    return
end

fjac (generic function with 1 method)

In [5]:
# initialize
u0 = zeros(20)
u0[2]  = 0.2
u0[4]  = 0.04
u0[7]  = 0.1
u0[8]  = 0.3
u0[9]  = 0.01
u0[17] = 0.007
@show u0;

u0 = [0.0, 0.2, 0.0, 0.04, 0.0, 0.0, 0.1, 0.3, 0.01, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.007, 0.0, 0.0, 0.0]


In [6]:
# solver settings
abstol = 1e-12
reltol = 1e-9
p_true = ones(25)
solver = DifferentialEquations.CVODE_BDF()
t_end = 1e1
t = 10 .^ range(log10(1e-2), log10(t_end), length=301);

In [7]:
# parameters
rng = MersenneTwister(0x7777777)
factor = exp.((rand(rng, length(p_true)) .* 2 .- 1) .* 1)
p_pred = p_true .* factor
params = Flux.params(p_pred);
@show p_pred;

p_pred = [1.6134509333384142, 1.5120442024665515, 1.4734661603611492, 1.9828797665042102, 0.9637846993162412, 0.9193173817841414, 1.9931334900091495, 0.7436006332139167, 0.7486903309697442, 0.41009308366594693, 0.7727850930711746, 0.8049994647400657, 1.2437395135576494, 0.4423116556319156, 0.8877652479995815, 0.5795456004767549, 0.5089367394039388, 0.8380787764547069, 0.8821158618847272, 1.0092668395620195, 0.5553157380025299, 0.7828301951691133, 0.6262498295589252, 0.4609633962339217, 0.8485644653887581]


In [8]:
# solve the ODE
prob = ODEProblem(ODEFunction(f, jac=fjac), u0, (0.0,t_end));
y_true = solve(prob, solver, p=p_true, abstol=abstol, reltol=reltol, saveat=t);

In [9]:
# define the problem
function predict_rd()
    y_pred = solve(prob, solver, p=p_pred, abstol=abstol, reltol=reltol, saveat=t)
end

predict_rd (generic function with 1 method)

In [12]:
function loss_1()
    sum(abs2, predict_rd()-y_true)
end

@time Flux.gradient(()->loss_1(), params);

  0.327882 seconds (1.63 M allocations: 66.166 MiB)


In [13]:
weights = ones(20).*1e3

function loss_2()
    sum(abs2, (predict_rd()-y_true).*weights)
end

@time Flux.gradient(()->loss_2(), params);

  4.565923 seconds (46.90 M allocations: 1.889 GiB, 6.52% gc time)



[CVODES WARNING]  CVode
  Internal t = 10 and h = -8.51646e-18 are such that t + h = t on the next step. The solver will continue anyway.


[CVODES WARNING]  CVode
  Internal t = 10 and h = -3.08008e-16 are such that t + h = t on the next step. The solver will continue anyway.


[CVODES WARNING]  CVode
  Internal t = 10 and h = -7.2163e-16 are such that t + h = t on the next step. The solver will continue anyway.


[CVODES WARNING]  CVode
  Internal t = 10 and h = -7.94248e-16 are such that t + h = t on the next step. The solver will continue anyway.


[CVODES WARNING]  CVode
  Internal t = 10 and h = -8.53323e-16 are such that t + h = t on the next step. The solver will continue anyway.


[CVODES WARNING]  CVode
  Internal t = 10 and h = -3.03935e-19 are such that t + h = t on the next step. The solver will continue anyway.


[CVODES WARNING]  CVode
  Internal t = 10 and h = -3.03935e-19 are such that t + h = t on the next step. The solver will continue anyway.


[CVODES WARNING]  CV